In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [ ]:
# read multiple csv from data folder
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(file)
print(files)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
path_to_germany = "./data/vg2500_geo84/vg2500_bld.shp"
germany_gdf = gpd.read_file(path_to_germany)
germany_gdf.plot()

## Plot germany with grid

In [ ]:
df = pd.read_csv('data/' + files[0])

In [ ]:
df

In [ ]:
geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
geo_df = gpd.GeoDataFrame(df, geometry=geometry)

In [ ]:
fig, ax = plt.subplots()
germany_gdf.plot(ax=ax, color='lightgrey')

geo_df.plot(ax=ax, marker='o', color='red', markersize=5)

plt.show()

### Seperate the map of germany into a grid 

In [ ]:
# Calculate midpoints
mid_latitude = df['latitude'].mean()
mid_longitude = df['longitude'].mean()

def categorize_location(row):
    if row['latitude'] >= mid_latitude and row['longitude'] <= mid_longitude:
        return 'top_left'
    elif row['latitude'] >= mid_latitude and row['longitude'] > mid_longitude:
        return 'top_right'
    elif row['latitude'] < mid_latitude and row['longitude'] <= mid_longitude:
        return 'bottom_left'
    else:
        return 'bottom_right'

# Apply the function to create the new 'location' column
df['location'] = df.apply(categorize_location, axis=1)
df['location']




In [ ]:
df

In [ ]:
# plot each location seperately
fig, ax = plt.subplots()
for i in df['location'].unique():
    temp_df = df[df['location'] == i]
    ax.scatter(temp_df['longitude'], temp_df['latitude'], label=i)
ax.legend()
plt.show()

## Clean dataframe with only important columns

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=["blh","tcc", "tsr", "sund", "tp", "fsr", "cdir", "z", "msl"])
df.columns

## Read measurements

In [ ]:
df_realized_supply = pd.read_csv('data/' + files[2], sep=';')
df_realized_supply.columns

### Again, drop unnecessary columns

In [ ]:
df_realized_supply = df_realized_supply[['Date from', 'Date to', "Photovoltaic [MW]", "Wind Offshore [MW] ", "Wind Onshore [MW]"]]

In [ ]:
df_realized_supply["wind_on_offshore"] = df_realized_supply["Wind Offshore [MW] "] + df_realized_supply["Wind Onshore [MW]"]

In [ ]:
df_realized_supply = df_realized_supply.drop(columns=["Wind Offshore [MW] ", "Wind Onshore [MW]"])

In [ ]:
df_realized_supply["photo"] = df_realized_supply["Photovoltaic [MW]"]

In [ ]:
df_realized_supply = df_realized_supply.drop(columns=["Photovoltaic [MW]"])

In [ ]:
df_realized_supply

## Plot measurements

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
fig = go.Figure()
x_axis = df_realized_supply["Date from"]

fig  = px.line(x=x_axis, y=df_realized_supply.photo,
                    )
fig.show()



In [ ]:
fig = go.Figure()
x_axis = df_realized_supply["Date from"]

fig  = px.line(x=x_axis, y=df_realized_supply.wind_on_offshore,
                    )
fig.show()



## Get year and month from date

In [ ]:
df_realized_supply["timestamps"] = pd.to_datetime(df_realized_supply["Date from"])
df_realized_supply['month_year'] = df_realized_supply['timestamps'].dt.strftime('%Y-%m')
df_realized_supply['day'] = df_realized_supply['timestamps'].dt.strftime('%d')
df_realized_supply["fullhour"] = df_realized_supply['timestamps'].dt.strftime('%H:%M')

In [ ]:
df_realized_supply

In [ ]:
# take every 4th row
df_full_hour = df_realized_supply.iloc[::4, :]
df_full_hour

In [ ]:
df_full_4_hours = df_realized_supply.iloc[::16, :]
df_full_4_hours

In [ ]:
df_full_6_hours = df_realized_supply.iloc[::24, :]
df_full_6_hours

In [ ]:
def preprocess_ssr(value):
    # Remove everything after the comma
    if type(value) != float:
        value = value.split(',')[0]
        # Remove any periods that are used as thousand separators
        value = value.replace('.', '')
        # Convert to float
    return float(value)
df_realized_supply["photo"] = df_realized_supply["photo"].apply(preprocess_ssr)
df_realized_supply["wind_on_offshore"] = df_realized_supply["wind_on_offshore"].apply(preprocess_ssr)

df_full_hour["photo"] = df_realized_supply["photo"].apply(preprocess_ssr)
df_full_hour["wind_on_offshore"] = df_realized_supply["wind_on_offshore"].apply(preprocess_ssr)




In [ ]:

df_full_4_hours["photo"] = df_realized_supply["photo"].apply(preprocess_ssr)
df_full_4_hours["wind_on_offshore"] = df_realized_supply["wind_on_offshore"].apply(preprocess_ssr)


In [ ]:
df_agg = df_realized_supply.groupby('month_year')["photo"].mean().reset_index()
fig = go.Figure()
x_axis = df_agg["month_year"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per month",
    xaxis_title="Month",
    yaxis_title="Power supply [MW]",
)
fig.show()



## Take weakly average

In [ ]:
df_hourly = df_realized_supply.resample('H', on="timestamps").photo.mean().reset_index()

df_daily = df_hourly.resample('D', on="timestamps").photo.mean().reset_index()

df_weekly = df_daily.resample('W', on='timestamps').photo.mean().reset_index()


In [ ]:
df_hourly.sort_values(by="timestamps", inplace=True)
df_agg = df_hourly
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per hour",
    xaxis_title="hour",
    yaxis_title="Power supply [MW]",
)
fig.show()

In [ ]:
df_weekly.sort_values(by="timestamps", inplace=True)
df_agg = df_weekly
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per hour",
    xaxis_title="week",
    yaxis_title="Power supply [MW]",
)
fig.show()

In [ ]:
df_train


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

df_final = df_weekly
df_train = df_final[df_final["timestamps"] < "2022-01-01"]
df_test = df_final[df_final["timestamps"]>="2022-01-01"]

values = df_train['photo']
values_test = df_test['photo']

# Fit a state space model using SARIMAX
model = SARIMAX(values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))  # Monthly seasonality
fit = model.fit(disp=False)

# Get the filtered state estimates
filtered_state_means = fit.filter_results.filtered_state[0]

# Convert the filtered values to a Pandas series
filtered_series = pd.Series(filtered_state_means, df_train["timestamps"])
state_transition_matrix = fit.filter_results.transition
observation_matrix = fit.filter_results.design
process_covariance_matrix = fit.filter_results.state_cov
measurement_covariance_matrix = fit.filter_results.obs_cov
initial_state_mean = fit.filter_results.initial_state
initial_state_covariance = fit.filter_results.initial_state_cov

# Display the properties
print("State Transition Matrix (F):")
print(state_transition_matrix)
print("\nObservation Matrix (H):")
print(observation_matrix)
print("\nProcess Covariance Matrix (Q):")
print(process_covariance_matrix)
print("\nMeasurement Covariance Matrix (R):")
print(measurement_covariance_matrix)
print("\nInitial State Mean:")
print(initial_state_mean)
print("\nInitial State Covariance:")
print(initial_state_covariance)


# Plot the original and filtered time series
plt.figure(figsize=(15, 5))
plt.plot(df_train["timestamps"], values, label='Original')
plt.plot(filtered_series.index, filtered_series, label='Filtered')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filtered Time Series')
plt.show()


predictions_test = fit.get_forecast(steps=len(df_test))
predicted_mean_test = predictions_test.predicted_mean

weekly_pred_df = pd.DataFrame({'predicted_mean': predicted_mean_test})
weekly_pred_df.index = df_test["timestamps"]
# Resample to daily frequency and interpolate
hourly_predictions = weekly_pred_df.resample('H').interpolate()


# Plot the original and predicted time series
plt.figure(figsize=(15, 5))
plt.plot(df_train["timestamps"], values, label='Train')
plt.plot(df_test["timestamps"], values_test, label='Test')
plt.plot(hourly_predictions.index, hourly_predictions, label='hourly Predictions')
plt.plot(df_test["timestamps"], predicted_mean_test, label='Predicted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Original vs Predicted Time Series')
plt.show()


In [ ]:
plt.plot(hourly_predictions.index, hourly_predictions, label='Daily Predictions')

In [ ]:
hourly_predictions

In [ ]:
weekly_pred_df = pd.DataFrame({'predicted_mean': predicted_mean_test})
weekly_pred_df.index = df_test["timestamps"]
# Resample to daily frequency and interpolate
daily_predictions = weekly_pred_df.resample('D').interpolate()

In [ ]:
daily_predictions

In [ ]:
plt.plot(daily_predictions.index, daily_predictions, label='Daily Predictions')

In [ ]:
df_hourly

In [ ]:
df_hourly_certain_time = df_hourly[df_hourly["timestamps"]>="2022-01-02"]

In [ ]:
plt.plot(df_hourly_certain_time["timestamps"], df_hourly_certain_time["photo"], label='Hourly vals')
plt.plot(hourly_predictions.index, hourly_predictions, label='hourly Predictions')

In [ ]:
df_hourly_certain_time

In [ ]:
hourly_predictions

In [ ]:
hourly_index_2 = pd.date_range(start=df_weekly["timestamps"].min(), end=df_weekly["timestamps"].max(), freq='H')
hourly_index_2

In [ ]:
# Generate an hourly date range that covers the period of the original hourly data
hourly_index = pd.date_range(start=df_weekly.index.min(), end=df_weekly.index.max(), freq='H')


# Reindex to ensure hourly_predictions covers the entire hourly_index range
hourly_predictions_2 = hourly_predictions.reindex(hourly_index_2).interpolate()
hourly_predictions_2



In [ ]:
# Limit the size to prevent excessive plotting issues
hourly_predictions_3 = hourly_predictions_2.loc[hourly_predictions.index.min():hourly_predictions.index.max()]

In [ ]:
# Assuming df_hourly is your hourly data DataFrame
df_hourly['timestamps'] = pd.to_datetime(df_hourly['timestamps'])
df_hourly.set_index('timestamps', inplace=True)

In [ ]:
hourly_predictions_3 = hourly_predictions_3[hourly_predictions_3.index.isin(df_hourly.index)] 

In [ ]:
hourly_predictions_3

In [ ]:


# Align hourly_predictions with the original hourly data
# hourly_predictions = hourly_predictions.reindex(df_hourly.index).interpolate()

# Calculate residuals on the training set
residuals_train = df_hourly.loc[hourly_predictions_3.index, 'photo'] - hourly_predictions_3.loc[hourly_predictions_3.index, 'predicted_mean']

# Plot residuals
plt.figure(figsize=(15, 5))
plt.plot(residuals_train.index, residuals_train, label='Residuals')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.title('Residuals from SARIMAX Model')
plt.show()


In [ ]:
residuals_train.fillna(0, inplace=True)

In [ ]:
# Initialize the Kalman Filter with better-informed parameters
from pykalman import KalmanFilter
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=residuals_train.mean(),
    initial_state_covariance=np.var(residuals_train),
    observation_covariance=np.var(residuals_train),
    transition_covariance=np.eye(1) * 0.01
)

# Fit the Kalman Filter on the residuals
kf_state_means, kf_state_covariances = kf.smooth(residuals_train.values)

# Convert the filtered values to a Pandas series
kalman_filtered_residuals = pd.Series(kf_state_means.flatten(), index=residuals_train.index)


In [ ]:
kf_state_means

In [ ]:
# Combine the hourly predictions with the Kalman filter output
combined_predictions = hourly_predictions_3['predicted_mean'] + kalman_filtered_residuals.reindex(hourly_predictions_3.index, method='nearest')

# Limit the size to prevent excessive plotting issues
# combined_predictions = combined_predictions.loc[df_hourly.index.min():df_hourly.index.max()]

# Plot the original, hourly, and combined predictions
plt.figure(figsize=(15, 5))
plt.plot(df_hourly.index, df_hourly['photo'], label='Original')
plt.plot(hourly_predictions_3.index, hourly_predictions_3, label='Hourly Predictions')
plt.plot(combined_predictions.index, combined_predictions, label='Combined Predictions')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Original vs Hourly vs Combined Predictions')
plt.show()


In [ ]:
combined_predictions

In [ ]:
df_full_hour.sort_values(by="timestamps", inplace=True)
df_agg = df_full_hour
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per hour",
    xaxis_title="week",
    yaxis_title="Power supply [MW]",
)
fig.show()

In [ ]:
df_train = df_weekly[df_weekly["timestamps"] < "2022-01-01"]
df_test = df_weekly[df_weekly["timestamps"] >= "2022-01-01"]

values_train = df_train['photo']
values_test = df_test['photo']
values_train.fillna(0, inplace=True)
kf = KalmanFilter(
    n_dim_obs=1,  # Dimension of observations (1 because we have a single PV output variable)
    n_dim_state=2,  # Dimension of state variables (level and trend)
    em_vars=['transition_matrices', 'observation_matrices', 'transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
)

# Use the EM algorithm to estimate parameters from the training data
kf = kf.em(values_train, n_iter=150)


In [ ]:
df_train["timstamp"] = pd.to_datetime(df_train["timestamps"])
df_train.set_index("timstamp", inplace=True)


In [ ]:
# Apply the Kalman filter to smooth the data
(smoothed_state_means_train, smoothed_state_covariances_train) = kf.smooth(values_train)


In [ ]:
smoothed_state_means_train.shape

In [ ]:
# Convert the smoothed state means to a Pandas series
smoothed_series = pd.Series(smoothed_state_means_train[:, 0], index=df_train.index)  # Use the first state variable (level)

# Plot the original and smoothed time series
plt.figure(figsize=(15, 5))
plt.plot(df_train.index, values_train, label='Original')
plt.plot(df_train.index, smoothed_series, label='Smoothed')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Original vs Smoothed Time Series')
plt.show()


In [ ]:
# Predict the next steps using the last state from the training data
n_test_steps = len(values_test)
predicted_means = []

# Initialize state with the last smoothed state from training
current_state_mean = smoothed_state_means_train[-1]
current_state_covariance = smoothed_state_covariances_train[-1]

for t in range(n_test_steps):
    # Predict the next step
    current_state_mean, current_state_covariance = kf.filter_update(
        current_state_mean, current_state_covariance, observation=None, transition_matrix=kf.transition_matrices, observation_matrix=kf.observation_matrices, transition_covariance=kf.transition_covariance,
    )
    predicted_means.append(current_state_mean[0])

predicted_means = np.array(predicted_means)


In [ ]:
print("Transition matrix:\n", kf.transition_matrices)
print("Observation matrix:\n", kf.observation_matrices)
print("Transition covariance:\n", kf.transition_covariance)
print("Observation covariance:\n", kf.observation_covariance)
print("Initial state mean:\n", kf.initial_state_mean)
print("Initial state covariance:\n", kf.initial_state_covariance)


# Print the initial state used for predictions
print("Initial state mean for predictions:\n", initial_state_mean)
print("Initial state covariance for predictions:\n", initial_state_covariance)



In [ ]:
# check if array has na
values_test.fillna(0, inplace=True)
np.isnan(predicted_means).any()

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Evaluate the predictions
r2 = r2_score(values_test, predicted_means)
mse = mean_squared_error(values_test, predicted_means)


print(f'R^2: {r2:.4f}')
print(f'MSE: {mse:.4f}')

# Convert the predicted values to a Pandas series for plotting
predicted_series = pd.Series(predicted_means, index=df_test["timestamps"])

# Plot the original and predicted time series
plt.figure(figsize=(15, 5))
# plt.plot(df_train.index, df_train['photo'], label='Train')
# plt.plot(df_test.index, df_test['photo'], label='Test')
plt.plot(predicted_series.index, predicted_series, label='Predicted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Original vs Predicted Time Series')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
from sklearn.metrics import r2_score, mean_squared_error

# Step 1: Generate Synthetic Data
np.random.seed(42)  # For reproducibility

# Generate a date range for one year of hourly data
date_range = pd.date_range(start='2020-01-01', end='2020-12-31 23:00:00', freq='H')

# Generate synthetic data: a combination of a sinusoidal pattern and random noise
sin_pattern = np.sin(np.linspace(0, 2 * np.pi * 4, len(date_range))) * 100  # Sinusoidal pattern (4 cycles per year)
trend = np.linspace(50, 150, len(date_range))  # Linear trend
noise = np.random.normal(0, 10, len(date_range))  # Random noise
synthetic_data = sin_pattern + trend + noise

# Create a DataFrame
df = pd.DataFrame({'timestamps': date_range, 'photo': synthetic_data})
df.set_index('timestamps', inplace=True)

# Split data into training and testing sets
split_date = '2020-10-01'
df_train = df[df.index < split_date]
df_test = df[df.index >= split_date]

values_train = df_train['photo'].values
values_test = df_test['photo'].values

# Step 2: Initialize and Train the Kalman Filter with EM Algorithm
kf = KalmanFilter(
    n_dim_obs=1,  # Dimension of observations (1 because we have a single PV output variable)
    n_dim_state=2,  # Dimension of state variables (level and trend)
    em_vars=['transition_matrices', 'observation_matrices', 'transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
)

kf = kf.em(values_train, n_iter=20)

# Step 3: Smooth the Training Data
(smoothed_state_means_train, smoothed_state_covariances_train) = kf.smooth(values_train)

# Plot the original and smoothed time series
smoothed_series = pd.Series(smoothed_state_means_train[:, 0], index=df_train.index)

plt.figure(figsize=(15, 5))
plt.plot(df_train.index, values_train, label='Original')
plt.plot(smoothed_series.index, smoothed_series, label='Smoothed')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Original vs Smoothed Time Series')
plt.show()

# Step 4: Predict the Test Data
def predict_kalman_filter(kf, initial_state_mean, initial_state_covariance, n_steps):
    predicted_means = []
    current_state_mean = initial_state_mean
    current_state_covariance = initial_state_covariance
    
    for t in range(n_steps):
        current_state_mean, current_state_covariance = kf.filter_update(
            current_state_mean, current_state_covariance, observation=None
        )
        predicted_means.append(current_state_mean[0])
    
    return np.array(predicted_means)

initial_state_mean = smoothed_state_means_train[-1]
initial_state_covariance = smoothed_state_covariances_train[-1]
n_test_steps = len(values_test)

predicted_means = predict_kalman_filter(kf, initial_state_mean, initial_state_covariance, n_test_steps)

# Step 5: Evaluate and Plot the Results
r2 = r2_score(values_test, predicted_means)
mse = mean_squared_error(values_test, predicted_means)

print(f'R^2: {r2:.4f}')
print(f'MSE: {mse:.4f}')

predicted_series = pd.Series(predicted_means, index=df_test.index)

plt.figure(figsize=(15, 5))
plt.plot(df_train.index, df_train['photo'], label='Train')
plt.plot(df_test.index, df_test['photo'], label='Test')
plt.plot(predicted_series.index, predicted_series, label='Predicted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Original vs Predicted Time Series')
plt.show()

# Debugging: Print Kalman Filter Parameters
print("Transition matrix:\n", kf.transition_matrices)
print("Observation matrix:\n", kf.observation_matrices)
print("Transition covariance:\n", kf.transition_covariance)
print("Observation covariance:\n", kf.observation_covariance)
print("Initial state mean:\n", kf.initial_state_mean)
print("Initial state covariance:\n", kf.initial_state_covariance)
print("Initial state mean for predictions:\n", initial_state_mean)
print("Initial state covariance for predictions:\n", initial_state_covariance)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

# Generate synthetic time series data
np.random.seed(0)
time = np.linspace(0, 10, 500)
values = 100 + 10 * np.sin(time) + np.random.normal(size=time.shape) * 2

# Create a DataFrame
data = pd.DataFrame({'date': pd.date_range(start='2020-01-01', periods=len(time), freq='D'), 'value': values})

# Prepare the time series
values = data['value'].values

# Initial state mean
initial_state_mean = values[0]

# Observation matrix
observation_matrix = np.array([[1]])

# Transition matrix
transition_matrix = np.array([[1]])

# Process and observation noise covariance matrices
observation_covariance = np.diag([0.1])  # Tuning required
transition_covariance = np.diag([0.1])  # Tuning required

# Create the Kalman Filter instance
kf = KalmanFilter(
   n_dim_obs=1,  # Dimension of observations (1 because we have a single PV output variable)
    n_dim_state=2,  # Dimension of state variables (level and trend)
    em_vars=['transition_matrices', 'observation_matrices', 'transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
)

# Fit the filter
kf = kf.em(values, n_iter=10)  # EM algorithm to estimate the parameters

# Apply the filter to the data
filtered_state_means, filtered_state_covariances = kf.filter(values)
n_forecast = 50
last_filtered_state_mean = filtered_state_means[-1]
forecasted_state_means = last_filtered_state_mean
forecasted_values = [last_filtered_state_mean]

for _ in range(n_forecast):
    forecasted_state_means = np.dot(transition_matrix, forecasted_state_means)
    forecasted_values.append(forecasted_state_means[0])  # Ensure consistent shape

forecasted_values = np.array(forecasted_values)

# Plotting the results
plt.figure(figsize=(14, 7))
plt.plot(data['date'], values, label='Original')
plt.plot(data['date'], filtered_state_means, label='Filtered')
plt.plot(pd.date_range(start=data['date'].iloc[-1], periods=n_forecast + 1, freq='D')[1:], forecasted_values, label='Forecasted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filter - Original, Filtered and Forecasted Time Series')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

# Generate synthetic time series data
np.random.seed(0)
time = np.linspace(0, 10, 500)
values = 100 + 10 * np.sin(time) + np.random.normal(size=time.shape) * 2

# Create a DataFrame
data = pd.DataFrame({'date': pd.date_range(start='2020-01-01', periods=len(time), freq='D'), 'value': values})

# Prepare the time series
values = data['value'].values

# Create the Kalman Filter instance
kf = KalmanFilter(
    n_dim_obs=1,  # Dimension of observations (1 because we have a single PV output variable)
    n_dim_state=2,  # Dimension of state variables (level and trend)
    em_vars=['transition_matrices', 'observation_matrices', 'transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
)

# Initial guess of the state mean and covariance
initial_state_mean = [values[0], 0]
initial_state_covariance = np.eye(2)

# Fit the filter using the EM algorithm
kf = kf.em(values, n_iter=10)  # EM algorithm to estimate the parameters

# Apply the filter to the data
filtered_state_means, filtered_state_covariances = kf.filter(values.reshape(-1, 1))

# Forecast future values
n_forecast = 50
last_filtered_state_mean = filtered_state_means[-1]
forecasted_state_means = last_filtered_state_mean
forecasted_values = []
print(kf.transition_matrices)

for _ in range(n_forecast):
    forecasted_state_means = kf.transition_matrices @ forecasted_state_means
    forecasted_values.append(forecasted_state_means[0])

forecasted_values = np.array(forecasted_values)

# Plotting the results
plt.figure(figsize=(14, 7))
# plt.plot(data['date'], values, label='Original')
# plt.plot(data['date'], filtered_state_means[:, 0], label='Filtered')
plt.plot(pd.date_range(start=data['date'].iloc[-1], periods=n_forecast + 1, freq='D')[1:], forecasted_values, label='Forecasted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filter - Original, Filtered and Forecasted Time Series')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

# Generate synthetic time series data
np.random.seed(0)
time = np.linspace(0, 10, 500)
values = 100 + 10 * np.sin(time) + np.random.normal(size=time.shape) * 2

# Create a DataFrame
data = pd.DataFrame({'date': pd.date_range(start='2020-01-01', periods=len(time), freq='D'), 'value': values})

# Prepare the time series
values = data['value'].values

# Define the Kalman Filter parameters
period = 2 * np.pi  # Example period of the sinusoid
omega = 2 * np.pi / period  # Angular frequency

# Transition matrix for capturing sinusoidal patterns
transition_matrix = np.array([
    [1, 1, 0, 0],
    [0, 1, -omega, 0],
    [0, 0, np.cos(omega), -np.sin(omega)],
    [0, 0, np.sin(omega), np.cos(omega)]
])

# Observation matrix
observation_matrix = np.array([[1, 0, 1, 0]])

# Initial state and covariance
initial_state_mean = [values[0], 0, 0, 0]  # Initial state [position, velocity, sin_component, cos_component]
initial_state_covariance = np.eye(4)  # Initial state covariance

# Observation and transition covariance
observation_covariance = np.eye(1) * 0.1  # Observation noise
transition_covariance = np.eye(4) * 0.1  # Process noise

# Create the Kalman Filter instance
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    observation_covariance=observation_covariance,
    transition_covariance=transition_covariance
)

# Fit the filter using the EM algorithm
kf = kf.em(values.reshape(-1, 1), n_iter=140)  # EM algorithm to estimate the parameters

# Apply the filter to the data
filtered_state_means, filtered_state_covariances = kf.filter(values.reshape(-1, 1))

# Forecast future values
n_forecast = 50
last_filtered_state_mean = filtered_state_means[-1]
forecasted_state_means = last_filtered_state_mean
forecasted_values = []

for _ in range(n_forecast):
    forecasted_state_means = np.dot(kf.transition_matrices, forecasted_state_means)
    forecasted_values.append(forecasted_state_means[0])

forecasted_values = np.array(forecasted_values)

# Plotting the results
plt.figure(figsize=(14, 7))
plt.plot(data['date'], values, label='Original')
plt.plot(data['date'], filtered_state_means[:, 0], label='Filtered')
plt.plot(pd.date_range(start=data['date'].iloc[-1], periods=n_forecast + 1, freq='D')[1:], forecasted_values, label='Forecasted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filter - Original, Filtered and Forecasted Time Series')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

# Generate synthetic time series data
np.random.seed(0)
time = np.linspace(0, 10, 500)
values = 100 + 10 * np.sin(time) + np.random.normal(size=time.shape) * 2

# Create a DataFrame
data = pd.DataFrame({'date': pd.date_range(start='2020-01-01', periods=len(time), freq='D'), 'value': values})

# Prepare the time series
values = data['value'].values

# Define the Kalman Filter parameters
# More complex transition matrix to capture sinusoidal patterns
dt = 1  # Time step
omega = 2 * np.pi / len(time)  # Angular frequency

transition_matrix = np.array([
    [1, dt, 0.5 * dt**2],
    [0, 1, dt],
    [0, 0, 1]
])

observation_matrix = np.array([[1, 0, 0]])  # We observe the position only

# Initial state and covariance
initial_state_mean = [values[0], 0, 0]  # Initial position, velocity, and acceleration
initial_state_covariance = np.eye(3)  # Initial state covariance

# Observation and transition covariance
observation_covariance = np.eye(1) * 0.1  # Observation noise
transition_covariance = np.eye(3) * 0.1  # Process noise

# Create the Kalman Filter instance
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    observation_covariance=observation_covariance,
    transition_covariance=transition_covariance
)

# Fit the filter using the EM algorithm
kf = kf.em(values.reshape(-1, 1), n_iter=10)  # EM algorithm to estimate the parameters

# Apply the filter to the data
filtered_state_means, filtered_state_covariances = kf.filter(values.reshape(-1, 1))

# Forecast future values using filter_update
n_forecast = 50
last_state_mean = filtered_state_means[-1]
last_state_covariance = filtered_state_covariances[-1]
forecasted_values = []

for _ in range(n_forecast):
    last_state_mean, last_state_covariance = kf.filter_update(
        last_state_mean, last_state_covariance
    )
    forecasted_values.append(last_state_mean[0])

forecasted_values = np.array(forecasted_values)

# Plotting the results
plt.figure(figsize=(14, 7))
plt.plot(data['date'], values, label='Original')
plt.plot(data['date'], filtered_state_means[:, 0], label='Filtered')
plt.plot(pd.date_range(start=data['date'].iloc[-1], periods=n_forecast + 1, freq='D')[1:], forecasted_values, label='Forecasted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filter - Original, Filtered and Forecasted Time Series')
plt.show()


In [ ]:
df_hourly[df_hourly["photo"].isnull()]["photo"] = 0


In [ ]:
df_hourly.fillna(0, inplace=True)

In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
from darts.metrics import mape
from darts.metrics.metrics import rmse, mae, r2_score

In [ ]:
df_final = df_hourly
df_train = df_final[df_final["timestamps"] < "2022-01-01"]
df_test = df_final[df_final["timestamps"]>="2022-01-01"]


series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_test = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=80)
model.fit(series)

forecast = model.predict(len(series_test))
# Validation
pred_val = forecast
val_error = mape(forecast, series_test)
print(f'MAPE on validation set: {val_error:.2f}%')

eval = rmse(series_test, forecast)
eval_mae = mae(series_test, forecast)
r2 = r2_score(series_test, forecast)




print(eval)
print(eval_mae)
print(r2)



plt.figure(figsize=(10, 6))
series.plot(label='train')
series_test.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Generate example data
np.random.seed(42)  # For reproducibility
dates = pd.date_range(start='2019-01-01', periods=365 * 2, freq='D')  # 2 years of daily data
seasonal_pattern = 10 + 2 * np.sin(np.linspace(0, 3.14 * 2 * 4, len(dates)))  # Seasonal pattern
noise = np.random.normal(0, 1, len(dates))  # Noise

values = seasonal_pattern + noise
data = pd.DataFrame({'date': dates, 'value': values})
data.to_csv('sample_data.csv', index=False)

# Display the generated data
print(data.head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Generate example data
np.random.seed(42)  # For reproducibility
dates = pd.date_range(start='2019-01-01', periods=365 * 2, freq='D')  # 2 years of daily data
seasonal_pattern = 10 + 2 * np.sin(np.linspace(0, 3.14 * 2 * 4, len(dates)))  # Seasonal pattern
noise = np.random.normal(0, 1, len(dates))  # Noise

values = seasonal_pattern + noise
data = pd.DataFrame({'date': dates, 'value': values})
data.to_csv('sample_data.csv', index=False)

# Load the generated sample data
data = pd.read_csv('sample_data.csv', parse_dates=['date'], index_col='date')

# Extract the time series values
values = data['value']

# Fit a state space model using SARIMAX
model = SARIMAX(values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 30))  # Monthly seasonality
fit = model.fit(disp=False)

# Get the filtered state estimates
filtered_state_means = fit.filter_results.filtered_state[0]

# Convert the filtered values to a Pandas series
filtered_series = pd.Series(filtered_state_means, index=data.index)

# Plot the original and filtered time series
plt.figure(figsize=(15, 5))
plt.plot(data.index, values, label='Original')
plt.plot(filtered_series.index, filtered_series, label='Filtered')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filtered Time Series')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

df_final = df_hourly
df_train = df_final[df_final["timestamps"] < "2022-01-01"]
df_test = df_final[df_final["timestamps"]>="2022-01-01"]

values = df_train['photo']

# Fit a state space model using SARIMAX
model = SARIMAX(values, order=(1, 1, 1), seasonal_order=(1, 1, 1, 30))  # Monthly seasonality
fit = model.fit(disp=True, maxiter=10)

# Get the filtered state estimates
filtered_state_means = fit.filter_results.filtered_state[0]

# Convert the filtered values to a Pandas series
filtered_series = pd.Series(filtered_state_means, index=data.index)

# Plot the original and filtered time series
plt.figure(figsize=(15, 5))
plt.plot(data.index, values, label='Original')
plt.plot(filtered_series.index, filtered_series, label='Filtered')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Kalman Filtered Time Series')
plt.show()


In [ ]:
df_agg = df_daily
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per day (actual)",
    xaxis_title="week",
    yaxis_title="Power supply [MW]",
)
fig.show()



In [ ]:
df_weekly.sort_values(by="timestamps", inplace=True)
df_agg = df_weekly
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per week (actual)",
    xaxis_title="week",
    yaxis_title="Power supply [MW]",
)
fig.show()



## Combine ssr with this

In [ ]:
pd.set_option('display.max_rows', 10)


In [ ]:
df = df.drop(columns=["longitude", "latitude"])

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
df["timestamps"] = pd.to_datetime(df["time"])

In [ ]:
df_hourly_ssr = df.resample('H', on="timestamps")["ssr"].mean().reset_index()
df_daily_ssr = df_hourly_ssr.resample('D', on="timestamps")["ssr"].mean().reset_index()
df_weekly_ssr =df_daily_ssr.resample('W', on="timestamps")["ssr"].mean().reset_index()
df_weekly_ssr


## join dataframes

In [ ]:
# df_joined = pd.merge(df_daily, df_daily_ssr, on="timestamps", how="inner")
df_joined = df_daily
df_joined

### Split training set

In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
from darts.datasets import AirPassengersDataset
import matplotlib.pyplot as plt
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics.metrics import rmse, mae, r2_score


In [ ]:
df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2021-01-01"]
df_test = df_final[df_final["timestamps"]>="2021-01-01"]

series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_actual = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=100
                         )
model.fit(series)

forecast = model.predict(365)

eval = rmse(series_actual, forecast)
eval_mae = mae(series_actual, forecast)
r2 = r2_score(series_actual, forecast)


print(eval)
print(eval_mae)
print(r2)
plt.figure(figsize=(10, 6))
series.plot(label='train')
series_actual.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:
df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2021-01-01"]
df_test = df_final[df_final["timestamps"]>="2021-01-01"]

series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_actual = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=200
                         )
model.fit(series)

forecast = model.predict(365)

eval = rmse(series_actual, forecast)
eval_mae = mae(series_actual, forecast)
r2 = r2_score(series_actual, forecast)


print(eval)
print(eval_mae)
print(r2)
plt.figure(figsize=(10, 6))
series.plot(label='train')
series_actual.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:
df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2021-01-01"]
df_test = df_final[df_final["timestamps"]>="2021-01-01"]

series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_actual = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=146
                         
                         )
model.fit(series)

forecast = model.predict(365)

eval = rmse(series_actual, forecast)
eval_mae = mae(series_actual, forecast)
r2 = r2_score(series_actual, forecast)


print(eval)
print(eval_mae)
print(r2)
plt.figure(figsize=(10, 6))
series.plot(label='train')
series_actual.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:
df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2021-01-01"]
df_test = df_final[df_final["timestamps"]>="2021-01-01"]

series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_actual = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=220
                         
                         )
model.fit(series)

forecast = model.predict(730)

eval = rmse(series_actual, forecast)
eval_mae = mae(series_actual, forecast)
r2 = r2_score(series_actual, forecast)


print(eval)
print(eval_mae)
print(r2)
plt.figure(figsize=(10, 6))
series.plot(label='train')
series_actual.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

## find good amount of hiddenstates

In [ ]:
import time

In [ ]:
df_joined

In [ ]:
df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2022-01-01"]
df_test = df_final[df_final["timestamps"]>="2022-01-01"]


In [ ]:
df_train

In [ ]:
print("start training..")
time_start = time.time()
series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_actual = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])
best_eval = 0
best_mae = 0
best_r2 = 0
best_number_states = 1
best_series = series

second_best_eval = 0
second_best_mae = 0
second_best_r2 = 0
second_best_number_states = 1

for i in range(220,285):

    model = KalmanForecaster(dim_x=i)
    model.fit(series)

    forecast = model.predict(365)

    eval = rmse(series_actual, forecast)
    eval_mae = mae(series_actual, forecast)
    r2 = r2_score(series_actual, forecast)
    if i%10==0:
        print(f"Step: {i}")
    if r2 > best_r2:


        second_best_eval = best_eval
        second_best_mae = best_mae
        second_best_r2 = best_r2
        second_best_number_states = best_number_states
        
        best_eval = eval
        best_mae = eval_mae
        best_r2 = r2
        best_number_states = i

        best_series = forecast
print("training finished")
print(f"duration:  {time.time()- time_start}")

print(f"RMSE: {best_eval}, MAE: {best_mae}, R2: {best_r2}, best_number_states: {best_number_states} \n")
print(f"Second: RMSE: {second_best_eval}, MAE: {second_best_mae}, R2: {second_best_r2}, best_number_states: {second_best_number_states}")
plt.figure(figsize=(10, 6))
series.plot(label='train')
series_actual.plot(label='test_vals')
best_series.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:


model = KalmanForecaster(dim_x=120)  # Specify the number of components (states)
model.fit(series)

# Forecast the next 20 time steps
forecast = model.predict(60)

# Plot the original series and the forecast
plt.figure(figsize=(10, 6))
series.plot(label='Actual')
forecast.plot(label='Forecast')
plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:
df_joined.columns

In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
from darts.datasets import AirPassengersDataset
import matplotlib.pyplot as plt
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from sklearn.preprocessing import StandardScaler

series = TimeSeries.from_dataframe(df_joined, 'timestamps', ['photo', "ssr"])

model = KalmanForecaster(dim_x=34)  # Specify the number of components (states)
model.fit(series)

# Forecast the next 20 time steps
forecast = model.predict(60)

# Plot the original series and the forecast
plt.figure(figsize=(10, 6))
series.plot(label='Actual')
forecast.plot(label='Forecast')
plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
from darts.datasets import AirPassengersDataset
import matplotlib.pyplot as plt
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_joined[['photo', 'ssr']] = scaler.fit_transform(df_joined[['photo', 'ssr']])
series = TimeSeries.from_dataframe(df_joined, 'timestamps', ['photo', "ssr"])

model = KalmanForecaster(dim_x=34)  # Specify the number of components (states)
model.fit(series)

# Forecast the next 20 time steps
forecast = model.predict(60)

# Plot the original series and the forecast
plt.figure(figsize=(10, 6))
series.plot(label='Actual')
forecast.plot(label='Forecast')
plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load your data


scaler = StandardScaler()
scaled_values = scaler.fit_transform(df_joined[['photo', 'ssr']])
df_joined[['photo', 'ssr']] = scaled_values

# Create a TimeSeries object with both 'photo' and 'ssr' columns
series = TimeSeries.from_dataframe(df_joined, 'timestamps', ['photo', 'ssr'])

# Initialize the KalmanForecaster with the appropriate dimension
model = KalmanForecaster(dim_x=34)  # Adjust dim_x based on the complexity needed
model.fit(series)

# Forecast the next 60 time steps
forecast = model.predict(60)

# Inverse transform the forecast data to the original scale
forecast_df = pd.DataFrame(forecast.pd_dataframe(), columns=['photo', 'ssr'])
forecast_inverse = scaler.inverse_transform(forecast_df)

# Reconstruct the TimeSeries object from the inverse transformed data
forecast_series = TimeSeries.from_dataframe(
    pd.DataFrame(forecast_inverse, index=forecast.time_index, columns=['photo', 'ssr']),
    time_col=None
)

# Plot the original series and the forecast
plt.figure(figsize=(10, 6))
series.plot(label='Actual')
forecast_series.plot(label='Forecast')
plt.legend()
plt.title('lol')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load your data


scaler = StandardScaler()
scaled_values = scaler.fit_transform(df_joined[['photo']])
df_joined[['photo']] = scaled_values

# Create a TimeSeries object with both 'photo' and 'ssr' columns
series = TimeSeries.from_dataframe(df_joined, 'timestamps', ['photo'])

# Initialize the KalmanForecaster with the appropriate dimension
model = KalmanForecaster(dim_x=34)  # Adjust dim_x based on the complexity needed
model.fit(series)

# Forecast the next 60 time steps
forecast = model.predict(60)

# Inverse transform the forecast data to the original scale
forecast_df = pd.DataFrame(forecast.pd_dataframe(), columns=['photo'])
forecast_inverse = scaler.inverse_transform(forecast_df)

# Reconstruct the TimeSeries object from the inverse transformed data
forecast_series = TimeSeries.from_dataframe(
    pd.DataFrame(forecast_inverse, index=forecast.time_index, columns=['photo']),
    time_col=None
)

# Plot the original series and the forecast
plt.figure(figsize=(10, 6))
series.plot(label='Actual')
forecast.plot(label='Forecast')
plt.legend()
plt.title('Forecast with only photovoltaic power supply')
plt.show()


In [ ]:
model

# Was wurde in der Masterarbeit gemacht ?
- Exponential Smoothing um Model zu erstellen
- Trend, seasonality und residual
- Dabei wurde in jedem Update step von Kalman auch die Parameter des Models geändert
- Parameter wurden mit Maximum Likelihood geschätzt
- Das Model als State Transition Model
- Die Messungen als Observation Model
- Das hat nur gut geklappt, weil das rausfinden des zugrundeliegenden Models durch die seasonalität und pattern möglich war
- hat auch autocorrelation genutzt um das window für die Tage zu finden - clever

## Problem für mich
- Exponential smoothing aufwändig
- Updaten von 2 Modellen so gesehen
- Auch rechenaufwändig (wie in der Masterarbeit beschrieben)
- Masterarbeitaufwand vs Seminararbeit 3 ects
- Bedarf kompletter Eigenimplementierung ohne Bibliothek

## Lösung
- Kalman verstanden
- Problemstellung verstanden
- Warum die Kombi nicht so gut ist in diesem Fall
- wann sie gut wäre (und was man machen müsste damit es hier gut ist)
- Nutze dennoch darts und erkläre N4SID
- Damit hätten wir:
    - State Space models
    - Kalman Filter
    - Usecases wo und wann er gut ist, was die einzelnen Komponenten sind
    - Vorgehen
    - Bezug auf unser Projekt, inwiefern das hier anwendbar ist
    - Lösung: N4SID und Kalman mittels Darts Implementierung
    - Fazit


## Fragen
- Macht es riesen Unterschied ob SSR und Photo oder nur Photo ?
- Multivariate vs Univariat ?
- Darts Implementierung etwas schwammig, hidden states nicht einsehbar, genauso wie die Kovarianzen - schlimm ?
Immerhin beschreibe ich ja was die jeweils machen und wie sie zusammenhängen

In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
from darts.metrics import mape

df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2021-06-01"]
df_val = df_final[(df_final["timestamps"]>="2021-06-01") & (df_final["timestamps"]<"2022-01-01")] 
df_test = df_final[df_final["timestamps"]>="2022-01-01"]

series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_val = TimeSeries.from_dataframe(df_val, "timestamps", ["photo"])


series_test = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=280)
model.fit(series)

forecast = model.predict(len(series_val))
# Validation
pred_val = forecast
val_error = mape(series_val, pred_val)
print(f'MAPE on validation set: {val_error:.2f}%')

eval = rmse(series_val, forecast)
eval_mae = mae(series_val, forecast)
r2 = r2_score(series_val, forecast)




print(eval)
print(eval_mae)
print(r2)

print("training again..")
combined_train_val = series.append(series_val)
model.fit(combined_train_val)
# Final testing
pred_test = model.predict(len(series_test))
test_error = mape(series_test, pred_test)
print(f'MAPE on test set: {test_error:.2f}%')
eval = rmse(pred_test, series_test)
eval_mae = mae(pred_test, series_test)
r2 = r2_score(pred_test, series_test)

print(eval)
print(eval_mae)
print(r2)


plt.figure(figsize=(10, 6))
combined_train_val.plot(label='train')
series_test.plot(label='test_vals')
pred_test.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()





In [ ]:
df_final = df_joined
df_train = df_final[df_final["timestamps"] < "2022-01-01"]
df_test = df_final[df_final["timestamps"]>="2022-01-01"]


series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_test = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=280)
model.fit(series)

forecast = model.predict(len(series_test))
# Validation
pred_val = forecast
val_error = mape(forecast, series_test)
print(f'MAPE on validation set: {val_error:.2f}%')

eval = rmse(series_test, forecast)
eval_mae = mae(series_test, forecast)
r2 = r2_score(series_test, forecast)




print(eval)
print(eval_mae)
print(r2)



plt.figure(figsize=(10, 6))
series.plot(label='train')
series_test.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()


In [ ]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# Example data (replace with your actual data)
np.random.seed(42)  # For reproducibility
n_timesteps = 100
solar_radiation = np.random.rand(n_timesteps)  # Simulated solar radiation data
pv_output = 2 * solar_radiation + np.random.randn(n_timesteps) * 0.1  # Simulated photovoltaic output data

# Normalize the data
solar_radiation = (solar_radiation - np.mean(solar_radiation)) / np.std(solar_radiation)
pv_output = (pv_output - np.mean(pv_output)) / np.std(pv_output)

# Dimensions
n_dim_state = 1  # We assume the state is one-dimensional (PV output)
n_dim_obs = 1  # We assume the observation is one-dimensional (solar radiation)

# Random initialization
transition_matrix = np.eye(n_dim_state)  # F: Identity matrix for state transition
observation_matrix = np.random.randn(n_dim_obs, n_dim_state)  # H: Random initialization
initial_state_mean = np.random.randn(n_dim_state)  # Initial state
initial_state_covariance = np.eye(n_dim_state)  # Initial state covariance
transition_covariance = np.eye(n_dim_state)  # Q: Process noise covariance
observation_covariance = np.eye(n_dim_obs)  # R: Measurement noise covariance
transition_offsets = np.zeros(n_dim_state)  # Transition offsets
observation_offset = np.zeros(n_dim_obs)  # Observation offsets

# Initialize Kalman Filter with random parameters
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    transition_covariance=transition_covariance,
    observation_covariance=observation_covariance,
    transition_offsets=transition_offsets,
    observation_offsets=observation_offset,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    em_vars=[
        'transition_matrices', 'observation_matrices',
        'transition_covariance', 'observation_covariance',
        'observation_offsets', 'initial_state_mean',
        'initial_state_covariance'
    ]
)

# Observations are the solar radiation data
observations = solar_radiation.reshape(-1, 1)

# Estimate parameters using EM algorithm
loglikelihoods = np.zeros(10)
for i in range(len(loglikelihoods)):
    kf = kf.em(X=observations, n_iter=1)
    loglikelihoods[i] = kf.loglikelihood(observations)

# Filtering
filtered_state_estimates = kf.filter(observations)[0]

# Plot results
pl.figure(figsize=(16, 6))
lines_obs = pl.plot(observations, linestyle='-', color='b', label='Solar Radiation (observations)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()

# Plot log likelihoods
pl.figure()
pl.plot(loglikelihoods)
pl.xlabel('EM iteration number')
pl.ylabel('Log likelihood')
pl.show()


In [ ]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# Example data (replace with your actual data)
np.random.seed(42)  # For reproducibility
n_timesteps = 100
solar_radiation = np.random.rand(n_timesteps)  # Simulated solar radiation data
pv_output = 2 * solar_radiation + np.random.randn(n_timesteps) * 0.1  # Simulated photovoltaic output data

# Normalize the data
solar_radiation = (solar_radiation - np.mean(solar_radiation)) / np.std(solar_radiation)
pv_output = (pv_output - np.mean(pv_output)) / np.std(pv_output)

# Prepare matrices
x = pv_output.reshape(-1, 1)  # Hidden states (PV output)
z = solar_radiation.reshape(-1, 1)  # Observations (solar radiation)

M = n_timesteps

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True PV Output (hidden state)')
lines_obs = pl.plot(z, linestyle=':', color='m', label='Solar Radiation (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()


In [ ]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# Example data (replace with your actual data)
np.random.seed(42)  # For reproducibility
n_timesteps = 100
solar_radiation = np.random.rand(n_timesteps)  # Simulated solar radiation data
pv_output = 2 * solar_radiation + np.random.randn(n_timesteps) * 0.1  # Simulated photovoltaic output data

# Normalize the data
solar_radiation = (solar_radiation - np.mean(solar_radiation)) / np.std(solar_radiation)
pv_output = (pv_output - np.mean(pv_output)) / np.std(pv_output)

# Prepare matrices
x = pv_output.reshape(-1, 1)  # Hidden states (PV output)
z = solar_radiation.reshape(-1, 1)  # Observations (solar radiation)

M = n_timesteps

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True PV Output (hidden state)')
lines_obs = pl.plot(z, linestyle=':', color='m', label='Solar Radiation (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()
